In [38]:
!nvidia-smi

Tue Mar 23 22:15:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    30W /  70W |   6090MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pretrainedmodels

# https://github.com/Cadene/pretrained-models.pytorch

     |████████████████████████████████| 61kB 5.1MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60963 sha256=2795afffbe7e762e9dcbfe30c2a1b253a4aba364dd5ce40cc3c6e4f23cc5e42b
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [16]:
import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

import pretrainedmodels

In [ ]:
batch_size = 8

In [ ]:
# setup
train_dataset = datasets.FashionMNIST(
  root='data',
  train=True,
  download=True,
  transform=transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor()]))

test_dataset = datasets.FashionMNIST(
  root='data',
  train=False,
  download=True,
    transform=transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor()]))


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
train_loader = DataLoader(
  train_dataset,
  batch_size=batch_size,
  shuffle=True,
  num_workers=2)

test_loader = DataLoader(
  test_dataset,
  batch_size=batch_size,
  num_workers=2)

In [ ]:
device = 'cuda'
print(len(train_dataset))
print(len(test_dataset))

60000
10000


In [ ]:
model = pretrainedmodels.__dict__['inceptionresnetv2'](num_classes=1000, pretrained='imagenet')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth" to /root/.cache/torch/hub/checkpoints/inceptionresnetv2-520b38e4.pth


In [33]:
class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, bias=False) # verify bias false
        self.bn = nn.BatchNorm2d(out_planes,
                                 eps=0.001, # value found in tensorflow
                                 momentum=0.1, # default pytorch value
                                 affine=True)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [34]:
model.conv2d_1a = BasicConv2d(1, 32, kernel_size=3, stride=2)
dim_feats = model.last_linear.in_features
nb_classes = 10
model.last_linear = nn.Linear(dim_feats, nb_classes)

In [35]:
model.to('cuda')

InceptionResNetV2(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [36]:
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss()
epochs = 3

In [39]:
#training loop
losses = []
for i in range(epochs):
  for j,(images,targets) in enumerate(tqdm.notebook.tqdm(train_loader)):
    images, targets = images.to('cuda'),targets.to('cuda')

    input = torch.autograd.Variable(images, requires_grad=False)

    #making predictions
    y_pred = model(images)
  
    #calculating loss
    loss = criterion(y_pred,targets.reshape(-1))
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(loss.item())
  losses.append(loss)


0.3273337483406067


KeyboardInterrupt: ignored

In [40]:
#accuracy of the model
total = 0
for  x_test,y_test in tqdm.notebook.tqdm(test_loader):
  x_test,y_test = x_test.to('cuda'),y_test.to('cuda')
  y_pred = model(x_test).argmax(dim=1)
  total += y_pred.eq(y_test).sum()
  
print("Accuracy is: ", (total/10000.).item()*100, "%")


Accuracy is:  92.25999712944031 %
